In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
from tensorflow import keras
from tensorflow.keras.layers import (Input, Layer, Embedding, Bidirectional,
                                     LSTM, Dense, Dropout, Conv1D, GlobalAveragePooling1D,
                                     LayerNormalization, MultiHeadAttention)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import layers
import numpy as np
import joblib

In [10]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [11]:
model = keras.models.load_model("/content/drive/MyDrive/fake_news/model.keras",
                                custom_objects = {"TransformerBlock": TransformerBlock})


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [15]:
tokenizer = joblib.load("/content/drive/MyDrive/fake_news/tokenizer.pkl")

In [16]:
MAX_LEN = 256

def predict(text):
  seq = tokenizer.texts_to_sequences([text])
  padded = pad_sequences(seq, maxlen = MAX_LEN)
  pred_prob = model.predict(padded)[0][0]

  if pred_prob >= 0.5:
    label = "REAL"
  else:
    label = "FAKE"

  return {"label" : label, "confidence": float(pred_prob)}

In [18]:
import gradio as gr

iface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=3, placeholder="Enter text here"),
    outputs=gr.JSON(),
    title="Fake News Classification",
    description="A deep learning–based fake news classification system that analyzes textual content and predicts whether a news article is Fake or Real."
)

iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://185e63e735a7f43d7c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
